In [ ]:
import pandas as pd
from joblib import Parallel, delayed

from os import listdir
from os.path import isfile, join

In [ ]:
cars, time_interval = 30, 40

In [ ]:
DATA_DIRECTORY = f"data/test_{cars}_{time_interval}_updated"

In [ ]:
def read_df(num):
    df = pd.read_csv(f"{DATA_DIRECTORY}/{num}.csv")
    
    with open(f"{DATA_DIRECTORY}/{num}.txt") as iter_file:
        df["iterations"] = int(iter_file.read())
        
    return df

In [ ]:
onlyfiles = [f for f in listdir(DATA_DIRECTORY) if isfile(join(DATA_DIRECTORY, f))]

In [ ]:
max_file = max(int(f.split('.')[0]) for f in onlyfiles)

In [ ]:
nums = range(max_file + 1)

In [ ]:
%%time
dfs = Parallel(n_jobs=10)(delayed(read_df)(num) for num in nums)

In [ ]:
all_together = pd.concat(dfs, axis=0).reset_index(drop=True)

In [ ]:
all_together

In [ ]:
data_cols = all_together.columns.drop(["iterations"])
groupby_stats =  all_together.groupby(list(data_cols))
avg_iterations = groupby_stats.mean()

In [ ]:
avg_iterations.reset_index(inplace=True)

In [ ]:
counts = groupby_stats.count().reset_index(drop=True)["iterations"]

In [ ]:
counts

In [ ]:
avg_iterations = avg_iterations.loc[avg_iterations.index.repeat(counts)].reset_index(drop=True)

In [ ]:
avg_iterations

In [ ]:
input_cols = avg_iterations.columns.drop(["used_algo", "iterations"])

In [ ]:
best_algos_index_vals = avg_iterations.groupby(list(input_cols))["iterations"].min().reset_index()

In [ ]:
best_algos_index_vals

In [ ]:
index_cols = [*list(input_cols), "iterations"]

best_algos_entries = [avg_iterations[(avg_iterations[index_cols] == best_algo).all(axis=1)] for _, best_algo in best_algos_index_vals.iterrows()]

In [ ]:
best_algos_df = pd.concat(best_algos_entries, axis=0)

In [ ]:
best_algos_df.reset_index(drop=True)

In [ ]:
output_path = f"data/test_{cars}_{time_interval}_model_data_updated.csv"

In [ ]:
best_algos_df.to_csv(output_path, index=False)